In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import FloatTensor as FT
from torch import LongTensor as LT
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable as V
import torch.cuda
import torch.optim as optim

# Hand written digit recognition with Resnet

In this notebook we will learn how to create Resnet with PyTorch use it on MNIST dataset. MNIST is collection of grayscale images of written digits with 10 classes (digits 0-9). The images have all the same resolution 28x28 pixels. We won't do any Exploratory Data Analysis, because the dataset shouldn't contain anything tricky (different sizes, channels,...).

Data: https://pjreddie.com/projects/mnist-in-csv/

### Data - PyTorch Dataset, Dataloader

We will use PyTorch Dataset and DataLoader classes for storing and iterating over data. The catch here is we won't directly store the training values in dataset. Instead, the Dataset object stores IDs and labels as a reference to the actual data. https://stanford.edu/~shervine/blog/pytorch-how-to-generate-data-parallel.html

In [18]:
PATH = 'data/'

In [177]:
mnist_train = pd.read_csv(f'{PATH}mnist_train.csv', header=None)
mnist_valid = pd.read_csv(f'{PATH}mnist_test.csv', header=None)

In [178]:
train_x = mnist_train.drop(columns=[0])
valid_x = mnist_valid.drop(columns=[0])
train_y = mnist_train.iloc[:,0]
valid_y = mnist_valid.iloc[:,0]
train_idx = mnist_train.index
valid_idx = mnist_valid.index

In [179]:
train_id_map = {idx:label for label, idx in zip(train_y, train_idx)}
valid_id_map = {idx:label for label, idx in zip(valid_y, valid_idx)}

In [180]:
class MNIST_Dataset(Dataset):
    def __init__(self, idxs, id_map, xs_df):
        self.id_map = id_map
        self.idxs = idxs
        self.xs_df = xs_df
        
    def __len__(self):
        return len(self.id_map)
    
    def __getitem__(self, idx):
        real_ID = self.idxs[idx]
        x = self.xs_df.iloc[real_ID]
        x = np.reshape(np.array(x),(28,28))
        y = self.id_map[real_ID]
        return x.astype(np.float32),y

In [181]:
train_dataset = MNIST_Dataset(train_idx,train_id_map,train_x)
valid_dataset = MNIST_Dataset(valid_idx,valid_id_map,valid_x)

In [182]:
bs = 64

In [183]:
train_dataloader = DataLoader(train_dataset, bs, True)
valid_dataloader = DataLoader(valid_dataset, bs)

### Model

Now that we have the data ready both for training and evaluation, we can proceed to create the ResNet itself.

In [87]:
class ConvWithBatchNorm(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.conv = nn.Conv2d(in_channels,out_channels,3,stride,padding=1)
        self.batch = nn.BatchNorm2d(out_channels)
        
    def forward(self, inp):
        out = self.conv(inp)
        return self.batch(F.relu(out))

In [88]:
class ResNetLayer(ConvWithBatchNorm):
    def forward(self, x): return x + super().forward(x)

In [157]:
class ResNet(nn.Module):
    def __init__(self, layers, classes, p=0.5):
        super().__init__()
        self.layers = layers
        self.conv1 = nn.Conv2d(in_channels=1,out_channels=layers[0],kernel_size=5,padding=2)
        self.bns = nn.ModuleList([ConvWithBatchNorm(layers[i], layers[i+1], stride=2) 
                                  for i in range(len(layers)-1)])
        self.res1s = nn.ModuleList([ResNetLayer(layers[i+1], layers[i+1]) for i in range(len(layers)-1)])
        self.res2s = nn.ModuleList([ResNetLayer(layers[i+1], layers[i+1]) for i in range(len(layers)-1)])
        self.out = nn.Linear(layers[-1], classes)
        self.drop = nn.Dropout(p)
        
    def forward(self, inp):
        out = self.conv1(inp)
        for b,r1,r2 in zip(self.bns, self.res1s, self.res2s):
            out = r2(r1(b(out)))
        out = F.adaptive_max_pool2d(out, 1)
        out = out.view(out.size(0), -1)
        out = self.drop(out)
        return self.out(out)

### Train loop

Here we will implement our own train function.

In [173]:
def train(m, train_dl, valid_dl, opt, loss_fn, epochs):
    def getLoss(valid=False, loss_fn=loss_fn):
        data = train_dl if not valid else valid_dl
        num_batch, loss_avg = 0,0
        m.eval()
        for x,y in data:
            x = V(x[:,None,:,:]).cuda()
            y = V(y).cuda()
            out = m(x)
            loss_avg += float(loss_fn(out,y))
            num_batch += 1
        m.train()
        return loss_avg/num_batch

    for e in range(epochs):
        for x,y in train_dl:
            # Add singleton dimension (channel)
            x = V(x[:,None,:,:]).cuda()
            y = V(y).cuda()
            out = m(x)
            opt.zero_grad()
            loss = loss_fn(out, y)
            loss.backward()
            opt.step()
        print(f'Epoch {e+1} \n \
                Train loss: {getLoss()} \n \
                Validation loss: {getLoss(True)} \n \
                Validation accuracy: {getLoss(True,accuracy)*100}%')

In [184]:
wd = 1e-5
m = ResNet([10,20,40,80,160],10).cuda()
opt = optim.Adam(m.parameters(), lr=1e-3, weight_decay=wd)
loss_fn = F.cross_entropy

In [185]:
def accuracy(pred,y):
    pred = pred.max(1)[1]
    acc = 0
    for i,p in enumerate(pred):
        acc += float((p == y[i]))
    return acc/len(pred)

In [186]:
train(m,train_dataloader,valid_dataloader,opt,loss_fn,20)

Epoch 1 
                 Train loss: 0.06263531989523216 
                 Validation loss: 0.06613515988941406 
                 Validation accuracy: 97.80055732484077%
Epoch 2 
                 Train loss: 0.0282764425521085 
                 Validation loss: 0.0344614742477988 
                 Validation accuracy: 98.86544585987261%
Epoch 3 
                 Train loss: 0.02393992072038813 
                 Validation loss: 0.03023368407301842 
                 Validation accuracy: 99.0047770700637%
Epoch 4 
                 Train loss: 0.02597112249114366 
                 Validation loss: 0.03985336769348497 
                 Validation accuracy: 98.82563694267516%
Epoch 5 
                 Train loss: 0.01897445623911838 
                 Validation loss: 0.033840199469760725 
                 Validation accuracy: 98.96496815286623%
Epoch 6 
                 Train loss: 0.021983704160350854 
                 Validation loss: 0.037374936162855976 
                 Validation acc